In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_datetime, utils
from phoenix.tag.topic import single_feature_match_topic_config as sfm_topic_config
from phoenix.tag.labeling.generate_label_sheet import create_new_labeling_sheet
from phoenix.tag import normalise

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_posts"
PIPELINE_ID = "test"

# OUTPUT
FINAL_URL_FACEBOOK_POSTS = None

In [ ]:
if RUN_DATETIME:
    run_dt = run_datetime.from_file_safe_str(RUN_DATETIME)
else:
    run_dt = run_datetime.create_run_datetime_now()
    
url_config = {
    "YEAR_FILTER": int(YEAR_FILTER),
    "MONTH_FILTER": int(MONTH_FILTER),
    "OBJECT_TYPE": OBJECT_TYPE,
}
art_url_reg = artifacts.registry.ArtifactURLRegistry(run_dt, ARTIFACTS_ENVIRONMENT_KEY)
TAGGING_RUNS_URL_FEATURES_FOR_TAGGING_FOLDER = art_url_reg.get_url("tagging_runs-features_for_tagging", url_config)
PIPELINE_TO_GOOGLE_SHEET_FOLDER_MAPPING = art_url_reg.get_url("static-g_sheet_folder_ids", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_FEATURES_FOR_TAGGING_FOLDER,
PIPELINE_TO_GOOGLE_SHEET_FOLDER_MAPPING,
run_dt.dt,
YEAR_FILTER,
MONTH_FILTER,
sep='\n',
)

In [ ]:
# This is placeholder code while waiting for the artifacts interface
PIPELINE_ID = "test"
with tentaclio.open(PIPELINE_TO_GOOGLE_SHEET_FOLDER_MAPPING) as fb:
    folder_mapping = pd.read_csv(fb)
    
g_sheet_folder_id = folder_mapping[folder_mapping["pipeline_id"] == PIPELINE_ID].iloc[0]["folder_id"]

In [ ]:
g_sheet_folder_id

In [ ]:
df = normalise.merge(TAGGING_RUNS_URL_FEATURES_FOR_TAGGING_FOLDER)

In [ ]:
df

In [ ]:
labeling_df = create_new_labeling_sheet_df(df)

In [ ]:
labeling_df.head()

In [ ]:
# temp usage of GoogleDriveInterface while the artifacts module is being developed
gdi = artifacts.google_drive.GoogleDriveInterface()

In [ ]:
sheet_mapping = gdi.get_files_in_folder(g_sheet_folder_id)

In [ ]:
sheet_id = sheet_mapping.get("test_sheet_phoenix")

In [ ]:
sheet_id

In [ ]:
gdi.update_sheet_with_df(df, sheet_id, clear_sheet=True)